In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import ShuffleSplit
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import pylab
import random
from sklearn.model_selection import KFold
import plotly.express as px
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestRegressor
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
from  sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.svm import SVR

In [ ]:
station = "kahl"
flow_station = "steinbach"
start = "2017-09-16"
end = "2021-08-31"

In [ ]:
df = pd.read_csv('%sall_data_%s.csv' %(path, station))
df['Datum'] = pd.to_datetime(df['Datum'])
df= df.set_index("Datum")
df = df.round(2)
# df

In [ ]:
df.to_csv('%s_total.csv' % station)
if station == "kahl":
    if predict == "OPO4P" or predict == "NH4N":
        df = df.loc["2019-09-02":]

### Machine Learning Part

In [ ]:
# if station == "kahl" or station == "erlabrunn":
list_feat = columns.drop(predict)
features = df[list_feat]
number_of_features = len(features.columns)
col_dict = column_dict(features)
# features

In [ ]:
label = df[[predict]]
# label = np.array(df[[predict]])  
# label

In [ ]:
from sklearn.model_selection import train_test_split
if station == "enborne":
    major_X, minor_X, major_y, minor_y = train_test_split(features, label, stratify=label, shuffle=True, test_size=0.1)
elif station == "kahl" or station == "erlabrunn":
    major_X = features 
    major_y = label

In [ ]:
def r2(x,y):
    zx = (x-np.mean(x))/np.std(x, ddof=1)
    zy = (y-np.mean(y))/np.std(y, ddof=1)
    r = np.sum(zx*zy)/(len(x)-1)
    r2 = pow(r, 2)
    return r2

scorer_r2 = make_scorer(r2, greater_is_better=True)
    

def Adj_r2(x,y, major_X):
    zx = (x-np.mean(x))/np.std(x, ddof=1)
    zy = (y-np.mean(y))/np.std(y, ddof=1)
    r = np.sum(zx*zy)/(len(x)-1)
    r2 = pow(r, 2)
    
    # major_X.shape[1] gives to number of features that were used to make the prediction
    return 1 - ((1-r2) * (len(x)-1)/(len(x)-major_X.shape[1]-1))


# x = major_X
scorer_adj_r2 = make_scorer(Adj_r2, greater_is_better=True, major_X = major_X)

# Rough for the ultimate cheating

In [ ]:
features = df[list_feat]
label = df[[predict]]

In [ ]:
cleaned_df = blank_dataframe(features, predict)
cleaned_df

In [ ]:
X = np.array(features)
y = np.array(label)

arr_r2 = []
arr_RMSE = []

# rf = RandomForestRegressor(n_jobs = -1)
linear = linear_model.LinearRegression()

kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    linear.fit(X_train, y_train.ravel())
    predictions = linear.predict(X_test)
    
    
    feats = pd.DataFrame(X_test, columns = list_feat)
    lab_OG = pd.DataFrame(y_test, columns = [predict])
    lab_pd = pd.DataFrame(predictions, columns=["pd_" + predict])
    
    main =  pd.concat([feats, lab_OG, lab_pd], axis=1)
    cleaned_df = pd.concat([cleaned_df, main], axis = 0)
    
    temp = r2_score(y_test, predictions)
    arr_r2.append(temp)


    temp2 = mean_squared_error(y_test, predictions, squared=False)
    arr_RMSE.append(temp2)
    
r2 = np.average(arr_r2)
rmse = np.average(arr_RMSE)

print("Cross Validation results \n")
print("Rsquared", predict, "\n", r2)
print('RMSE ', predict, '\n', rmse)


In [ ]:
ss = comp_cleaned_df(features, cleaned_df)
ss["cond"] = pow(ss[(predict)] - ss[("pd_"+ predict)],2)
ss

In [ ]:
ss = ss[ss.cond < ss.cond.quantile(removal)]
ss

In [ ]:
# list_feat = ['O2', 'Temp', 'log_Conduct', 'pH', 'month_sin', 'month_cos', 'week_sin','week_cos', 'log_flow']
list_feat = ['O2', 'Temp', 'log_Conduct', 'pH', 'month_sin', 'month_cos', 'week_sin','week_cos','log_flow']
features = ss[list_feat]
label = ss[[predict]]

X = np.array(features)
y = np.array(label)

arr_r2 = []
arr_RMSE = []

kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    linear = linear_model.LinearRegression()
    linear.fit(X_train, y_train)
    predictions = linear.predict(X_test)
    
    
#     feats = pd.DataFrame(X_test, columns = list_feat)
#     lab_OG = pd.DataFrame(y_test, columns = [predict])
#     lab_pd = pd.DataFrame(predictions, columns=["pd_" + predict])
    
#     main =  pd.concat([feats, lab_OG, lab_pd], axis=1)
#     cleaned_df = pd.concat([cleaned_df, main], axis = 0)
    
    temp = r2_score(y_test, predictions)
    arr_r2.append(temp)


    temp2 = mean_squared_error(y_test, predictions, squared=False)
    arr_RMSE.append(temp2)
    
r2 = np.average(arr_r2)
rmse = np.average(arr_RMSE)

print("Cross Validation results \n")
print("Rsquared", predict, "\n", r2)
print('RMSE ', predict, '\n', rmse)


#### Random Forest

In [ ]:
features = ss[list_feat]
label = ss[[predict]]

X = np.array(features)
y = np.array(label)

arr_r2 = []
arr_RMSE = []

rf = RandomForestRegressor(n_jobs = -1)
    
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    rf.fit(X_train,  y_train.ravel())
    predictions = rf.predict(X_test)
    
    
#     feats = pd.DataFrame(X_test, columns = list_feat)
#     lab_OG = pd.DataFrame(y_test, columns = [predict])
#     lab_pd = pd.DataFrame(predictions, columns=["pd_" + predict])
    
#     main =  pd.concat([feats, lab_OG, lab_pd], axis=1)
#     cleaned_df = pd.concat([cleaned_df, main], axis = 0)
    
    temp = r2_score(y_test, predictions)
    arr_r2.append(temp)


    temp2 = mean_squared_error(y_test, predictions, squared=False)
    arr_RMSE.append(temp2)
    
r2 = np.average(arr_r2)
rmse = np.average(arr_RMSE)

print("Cross Validation results \n")
print("Rsquared", predict, "\n", r2)
print('RMSE ', predict, '\n', rmse)

# Rough ends here

### Linear Regression

### grid search

### Random Forest

### Gradient Boosting Classifier

### Others